Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze_Assignment_1.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

### Blocked Rows and Columns:
[(14,9), (13,10), (12,10), (11,10), (10,10), (10,9), (9,9), (8,9), (7,9), (6,9), (11,12), (10,12), (11,13), (10,13)]

### Goal Loc:
(11,9)

### Init Loc:
(8,10)

### *All tuple written (row, col)

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

In [3]:

class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [4]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [5]:
from pprint import pprint
from collections import defaultdict

class Maze(Problem):
    #Your code goes here
    def __init__(self, rows, columns, initial=None, goal=None, **kwds):
        super().__init__(initial, goal, **kwds)
        self.cols = columns
        self.rows = rows
        self.blockers = defaultdict(list)
        
    def add_blocker(self,blocker):
        r,c = blocker
        self.blockers[r].append(c)
            
    def print_blockers(self):
        pprint(self.blockers)
        
    def actions(self, action, state): 
        r,c = state
        cost = 0
        if action == 'up':
            r += 1
            cost = 1
        if action == 'down':
            r -= 1
            cost = 1
        if action == 'left':
            c -= 1
            cost = 10
        if action == 'right':
            c += 1
            cost = 10
        if(r < 0 or c < 0 or r > self.rows or c > self.cols ):
            return {'state': state, 'cost': 0}
        if(c in self.blockers[r]):
            return {'state': state, 'cost': 0}
        return {'state':(r,c), 'cost': cost}
    

In [6]:
from pprint import pprint
maze = Maze(15,23, (8,10),(11,9))
blocker = [(14,9), (13,10), (12,10), (11,10), (10,10), (10,9), (9,9), (8,9), (7,9), (6,9), (11,12), (10,12), (11,13), (10,13)]

for b in blocker:
    maze.add_blocker(b)
    
    
pprint(maze.actions('right', (5,10)))

{'cost': 10, 'state': (5, 11)}


In [7]:
### BFS
from collections import deque

def bfs(maze):
    visited_nodes = defaultdict(list)
    actions = ['up', 'down', 'left', 'right']
    current_state = (8,10)
    visited_nodes[current_state[0]].append(current_state[1])

    frontier_node = Node(current_state)
    count_node = 1
    q = deque([frontier_node])
    frontier_count = len(q)
    max_frontier = 1


    while(len(q) != 0):
        expand_node = q.popleft()
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if(expand_node.state == s['state'] or (c in visited_nodes[r])): 
                continue
            frontier_node = Node(s['state'], expand_node, action, s['cost'])
            count_node += 1
            visited_nodes[r].append(c)
            q.append(frontier_node)
            
            if(maze.is_goal(frontier_node.state)):
                print('Goal reached!')
                pprint({'goal_node': frontier_node, 'max_frontier': max_frontier})
                return {'goal_node': frontier_node, 'max_frontier': max_frontier, 'count_node': count_node}
            

        frontier_count = len(q)
        max_frontier = max(max_frontier, frontier_count)
    #Goal not found    
    return False


solution = bfs(maze)

trace_node = solution['goal_node']
max_frontier = solution['max_frontier']
count_node = solution['count_node']
total_cost = 0
path = [trace_node]

while(trace_node.parent != None):
    total_cost += trace_node.path_cost
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])

Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 30}


Total cost:
39 

Maximum number of frontiers:
30 

Number of nodes created:
198 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [8]:
def is_cycle(expand_node, coord):
    curr = Node(None, expand_node)
    while(curr):
        if(curr.state == coord):
            return True
        curr = curr.parent
    return False
    

def dfs(maze):
    actions = ['up', 'right', 'down', 'left']
    current_state = (8,10)
    frontier_node = Node(current_state)
    
    stack = [frontier_node]
    frontier_count = len(stack)
    max_frontier = 1
    count_node = 1
    
    if maze.is_goal(frontier_node.state): 
        return {'frontier_node': frontier_node, 'max_frontier': max_frontier, 'count_node': count_node}
   
    while(len(stack)!= 0):
        expand_node = stack.pop()
        #print('expand', expand_node)
        #print(stack[:-3])
        if maze.is_goal(frontier_node.state): return {'goal_node': frontier_node, 'max_frontier': max_frontier, 'count_node': count_node}
        dirs = ''
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if (not is_cycle(expand_node, (r,c)) and expand_node.state != s['state']):
                dirs = action
                count_node += 1
                frontier_count = len(stack)
                max_frontier = max(max_frontier, frontier_count)
                frontier_node = Node(s['state'], expand_node, action, s['cost'])
                stack.append(frontier_node)
        
    return False


solution = dfs(maze)

print(solution)
trace_node = solution['goal_node']
max_frontier = solution['max_frontier']
count_node = solution['count_node']
total_cost = 0
path = [trace_node]

while(trace_node.parent != None):
    total_cost += trace_node.path_cost
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])

{'goal_node': <(11, 9)>, 'max_frontier': 306, 'count_node': 640}


Total cost:
2913 

Maximum number of frontiers:
306 

Number of nodes created:
640 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(5, 7)>, <(5, 6)>, <(5, 5)>, <(5, 4)>, <(5, 3)>, <(5, 2)>, <(5, 1)>, <(5, 0)>, <(4, 0)>, <(3, 0)>, <(2, 0)>, <(1, 0)>, <(0, 0)>, <(0, 1)>, <(0, 2)>, <(0, 3)>, <(0, 4)>, <(0, 5)>, <(0, 6)>, <(0, 7)>, <(0, 8)>, <(0, 9)>, <(0, 10)>, <(0, 11)>, <(0, 12)>, <(0, 13)>, <(0, 14)>, <(0, 15)>, <(0, 16)>, <(0, 17)>, <(0, 18)>, <(0, 19)>, <(0, 20)>, <(0, 21)>, <(0, 22)>, <(0, 23)>, <(1, 23)>, <(1, 22)>, <(1, 21)>, <(1, 20)>, <(1, 19)>, <(1, 18)>, <(1, 17)>, <(1, 16)>, <(1, 15)>, <(1, 14)>, <(1, 13)>, <(1, 12)>, <(1, 11)>, <(1, 10)>, <(1, 9)>, <(1, 8)>, <(1, 7)>, <(1, 6)>, <(1, 5)>, <(1, 4)>, <(1, 3)>, <(1, 2)>, <(1, 1)>, <(2, 1)>, <(2, 2)>, <(2, 3)>, <(2, 4)>, <(2, 5)>, <(2, 6)>, <(2, 7)>, <(2, 8)>, <(2, 9)>, <(2, 10)>, <(2, 11)>, <(2, 12)>, <(2, 13)>, <(2, 14)>, <(2, 15)>

In [9]:
### IDS with Cycle Check

## DLS
def dls(maze,l):

    actions = ['up', 'down', 'left', 'right']
    current_state = (8,10)
    
    frontier_node = Node(current_state)

    count_node = 1
    
    q = deque([frontier_node])

    frontier_count = len(q)
    max_frontier = 1


    while(len(q) != 0 ):
        
        expand_node = q.pop()
        
        if(maze.is_goal(expand_node.state)):
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}

        else:

            if len(expand_node)>l:
                continue
            
            for action in actions:
                s = maze.actions(action, expand_node.state)

                if(expand_node.state == s['state'] or is_cycle(expand_node, s['state'])): 
                    continue
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])
                    
                count_node += 1                 
                q.append(frontier_node)                

        frontier_count = len(q)        
        max_frontier = max(max_frontier, frontier_count)


    #Goal not found    
    return False

## IDS
def ids(maze):
    for i in range(0,99999):
        solution_dls=dls(maze,i)

        if solution_dls==False:
            print(f'Limit l={i}, no solution found. Continue with l={i+1}')
            
        else:
            print(f'Solution found with l={i}')
            return solution_dls
            
    
solution_ids=ids(maze)

trace_node = solution_ids['goal_node']
max_frontier = solution_ids['max_frontier']
count_node = solution_ids['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])


Limit l=0, no solution found. Continue with l=1
Limit l=1, no solution found. Continue with l=2
Limit l=2, no solution found. Continue with l=3
Limit l=3, no solution found. Continue with l=4
Limit l=4, no solution found. Continue with l=5
Limit l=5, no solution found. Continue with l=6
Limit l=6, no solution found. Continue with l=7
Limit l=7, no solution found. Continue with l=8
Limit l=8, no solution found. Continue with l=9
Limit l=9, no solution found. Continue with l=10
Limit l=10, no solution found. Continue with l=11
Goal reached!
{'count_node': 113494, 'goal_node': <(11, 9)>, 'max_frontier': 26}
Solution found with l=11


Total cost:
39 

Maximum number of frontiers:
26 

Number of nodes created:
113494 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [10]:
### Uniform Cost Search

from queue import PriorityQueue


def bestfs(maze):
    actions = ['up', 'left', 'down', 'right']
    q = PriorityQueue()
    current_state = (8,10)

    frontier_node = Node(current_state)
    
    #store visited notes in dictionary
    visited_nodes = defaultdict(tuple)

    visited_nodes[current_state] = frontier_node
    count_node = 1
    
    q.put((0, frontier_node))
    
    frontier_count = q.qsize()
    max_frontier = 1

    while(not q.empty()):
        current_cost, expand_node = q.get()
        
        
        if(maze.is_goal(expand_node.state) ):
            print(q.queue)
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}
          
        
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if(expand_node.state == s['state']): 
                continue

            visited_node = visited_nodes[(r,c)]
            
            if(not visited_node or visited_node.path_cost > expand_node.path_cost+s['cost']):
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])

                visited_nodes[r,c] = expand_node
                count_node += 1

                q.put((frontier_node.path_cost, frontier_node)) 
                              
        frontier_count = q.qsize()
        max_frontier = max(max_frontier, frontier_count)
        
    #Goal not found    
    return False


solution_ucs = bestfs(maze)


trace_node = solution_ucs['goal_node']
max_frontier = solution_ucs['max_frontier']
count_node = solution_ucs['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])


[(40, <(8, 14)>), (41, <(9, 14)>), (40, <(12, 7)>), (41, <(13, 7)>), (41, <(7, 14)>), (43, <(5, 14)>), (40, <(12, 9)>), (46, <(2, 6)>), (42, <(6, 14)>), (45, <(3, 14)>), (42, <(14, 7)>), (43, <(15, 7)>), (46, <(8, 6)>), (44, <(4, 14)>), (41, <(13, 9)>), (47, <(1, 6)>), (47, <(1, 14)>), (44, <(12, 14)>), (43, <(5, 6)>), (45, <(7, 6)>), (48, <(10, 6)>), (46, <(2, 14)>), (44, <(6, 6)>), (45, <(13, 14)>), (44, <(4, 6)>), (46, <(14, 14)>), (48, <(0, 14)>), (49, <(11, 6)>), (45, <(3, 6)>), (47, <(9, 6)>), (48, <(0, 6)>), (47, <(15, 14)>)]
Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 34}


Total cost:
39 

Maximum number of frontiers:
34 

Number of nodes created:
128 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [11]:
### A* search

from queue import PriorityQueue

def a_star(maze):
    actions = ['up', 'left', 'down', 'right']
    q = PriorityQueue()
    current_state = (8,10)
    goal_row = 11
    goal_col = 9
    frontier_node = Node(current_state)
    
    #store visited notes in dictionary
    visited_nodes = defaultdict(tuple)

    visited_nodes[current_state] = frontier_node
    count_node = 1
    
    q.put((0, frontier_node))
    
    frontier_count = q.qsize()
    max_frontier = 1

    while(not q.empty()):
        current_cost, expand_node = q.get()
        
        
        if(maze.is_goal(expand_node.state) ):
            print(q.queue)
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}
          
        
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if(expand_node.state == s['state']): 
                continue

            visited_node = visited_nodes[(r,c)]
            
            if(not visited_node or visited_node.path_cost > expand_node.path_cost+s['cost']):
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])

                visited_nodes[r,c] = expand_node
                count_node += 1

                heur = abs(r-goal_row) + abs(10*(c-goal_col))
                a_star_cost = heur + (frontier_node.path_cost)
                q.put((a_star_cost, frontier_node)) 
                              
        frontier_count = q.qsize()
        max_frontier = max(max_frontier, frontier_count)
        
    #Goal not found    
    return False


solution_ucs = a_star(maze)


trace_node = solution_ucs['goal_node']
max_frontier = solution_ucs['max_frontier']
count_node = solution_ucs['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])

[(41, <(4, 11)>), (41, <(15, 11)>), (41, <(15, 10)>), (47, <(1, 8)>), (41, <(4, 8)>), (41, <(12, 8)>), (45, <(2, 8)>), (55, <(7, 12)>), (49, <(0, 11)>), (53, <(8, 12)>), (43, <(3, 11)>), (45, <(2, 11)>), (43, <(3, 8)>), (49, <(0, 8)>), (47, <(1, 11)>), (59, <(5, 12)>), (55, <(12, 12)>), (57, <(13, 12)>), (53, <(9, 12)>), (59, <(14, 12)>), (59, <(5, 7)>), (59, <(6, 7)>), (59, <(7, 7)>), (59, <(8, 7)>), (59, <(9, 7)>), (59, <(10, 7)>), (57, <(6, 12)>), (59, <(11, 7)>)]
Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 29}


Total cost:
39 

Maximum number of frontiers:
29 

Number of nodes created:
67 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [12]:
from queue import PriorityQueue

def greedy_bfs(maze):
    actions = ['up', 'left', 'down', 'right']
    q = PriorityQueue()
    current_state = (8,10)
    goal_row = 11
    goal_col = 9
    frontier_node = Node(current_state)
    
    #store visited notes in dictionary
    visited_nodes = defaultdict(tuple)

    visited_nodes[current_state] = frontier_node
    count_node = 1
    
    q.put((0, frontier_node))
    
    frontier_count = q.qsize()
    max_frontier = 1

    while(not q.empty()):
        current_cost, expand_node = q.get()
        
        
        if(maze.is_goal(expand_node.state) ):
            print(q.queue)
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}
          
        
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if(expand_node.state == s['state']): 
                continue

            visited_node = visited_nodes[(r,c)]
            
            if(not visited_node or visited_node.path_cost > expand_node.path_cost+s['cost']):
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])

                visited_nodes[r,c] = expand_node
                count_node += 1

                heur = abs(r-goal_row) + abs(10*(c-goal_col))
                a_star_cost = heur
                q.put((a_star_cost, frontier_node)) 
                              
        frontier_count = q.qsize()
        max_frontier = max(max_frontier, frontier_count)
        
    #Goal not found    
    return False


solution_ucs = greedy_bfs(maze)


trace_node = solution_ucs['goal_node']
max_frontier = solution_ucs['max_frontier']
count_node = solution_ucs['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])

[(11, <(12, 8)>), (17, <(4, 10)>), (18, <(3, 10)>), (20, <(1, 10)>), (17, <(4, 8)>), (19, <(2, 8)>), (19, <(2, 10)>), (21, <(0, 8)>), (23, <(8, 11)>), (23, <(8, 11)>), (18, <(3, 8)>), (20, <(11, 7)>), (26, <(5, 11)>), (20, <(1, 8)>), (24, <(7, 11)>), (21, <(0, 10)>), (26, <(5, 7)>), (25, <(6, 7)>), (25, <(6, 11)>), (24, <(7, 7)>), (23, <(8, 7)>), (22, <(9, 7)>), (21, <(10, 7)>), (22, <(9, 11)>)]
Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 25}


Total cost:
39 

Maximum number of frontiers:
25 

Number of nodes created:
46 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [13]:
import numpy as np
import matplotlib.pyplot as plt

# creating the dataset
data = {'BFS':39, 'DFS':2913, 'IDS':39,
        'Uniform Cost':39, 'A Star': , }
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Search Type")
plt.ylabel("Result cost to goal")
plt.title("Goal cost of different Search types")
plt.show()


# creating the dataset
data = {'BFS':30, 'DFS':306, 'IDS':26,
        'Uniform Cost':34}
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Search Type")
plt.ylabel("Max Frontiers")
plt.title("Goal cost of different Search types")
plt.show()

# creating the dataset
data = {'BFS':198, 'DFS':640, 'IDS':113494,
        'Uniform Cost':128}
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Search Type")
plt.ylabel("Total frontiers created")
plt.title("Goal cost of different Search types")
plt.show()

SyntaxError: invalid syntax (410631516.py, line 6)